In [136]:
## Importar las librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statistics import mode
import nltk
from nltk.corpus import stopwords
from langdetect import detect, LangDetectException
from googletrans import Translator, LANGUAGES
import string
from num2words import num2words
import re
from nltk.corpus import cess_esp
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
import stanza 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor



## Descarga la información relevante de la libreria nltk
nltk.download('punkt') # Tokenizador de palabras
nltk.download('cess_esp') # Corpus en español
nltk.download('stopwords') # Palabras vacias
nltk.download('wordnet') # Sinonimos
stanza.download('es')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cess_esp to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\esteb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-04-02 19:12:50 INFO: Downloaded file to C:\Users\esteb\stanza_resources\resources.json
2024-04-02 19:12:50 INFO: Downloading default packages for language: es (Spanish) ...
2024-04-02 19:12:52 INFO: File exists: C:\Users\esteb\stanza_resources\es\default.zip
2024-04-02 19:13:00 INFO: Finished downloading models and saved to C:\Users\esteb\stanza_resources


In [2]:
## Importar el dataset
filepath = 'tipo1_entrenamiento_estudiantes.csv'
dataset = pd.read_csv(filepath)

In [3]:
# Define las contracciones del español
contracciones = {
    "q": "que",
    "d": "de",
    "t": "te",
    "m": "me",
    "na": "nada",
    "toy": "estoy",
    "toy": "estoy",
    "sta": "esta",
    "tamos": "estamos",
    'verm': 'ver',
    "al": "a el",
    "del": "de el",
    "bn": "bien",}

In [4]:
# Da un listado de las palabras que no tienen mucha utilidad en español
spanish_stopwords = set(stopwords.words('spanish'))



# Entendimiento de los datos

In [5]:
#Muestra las primeras filas del dataset
dataset.head()

,Review,Class
0,Nos alojamos en una casa alquilada en la ciuda...,4
1,"La comida está bien, pero nada especial. Yo te...",3
2,"En mi opinión, no es una como muchos usuarios ...",3
3,esta curiosa forma que asemeja una silla de mo...,4
4,Lo mejor era la limonada. Me gusto la comida d...,2


In [6]:
# Revisa el idioma del dataset
analisis= dataset.copy()

# #Se define una función que detecta el idioma de un texto
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "Unknown"

# Apply the language detection function to the first few rows as a test
analisis['lenguaje']=dataset['Review'].apply(detect_language)
language_counts = analisis['lenguaje'].value_counts()
language_counts

lenguaje
es    7870
en       4
pt       1
Name: count, dtype: int64

In [7]:
# Muestra las filas en inglés

# Filter the dataset for reviews estimated to be in English
english_reviews = analisis[analisis['lenguaje'] == 'en']

# Show the filtered rows
english_reviews[['Review', 'lenguaje']]

,Review,lenguaje
743,A must when you're in Bogota and it's just in ...,en
1497,First time in Colombia and the Hotel and servi...,en
3368,My expectations about this hotel were high due...,en
4835,This mercado is just like every mercado that I...,en


In [8]:
# Muestra las filas en portugués

# Filter the dataset for reviews estimated to be in English
portuguese_reviews = analisis[analisis['lenguaje'] == 'pt']

# Show the filtered rows
portuguese_reviews[['Review', 'lenguaje']]

,Review,lenguaje
1932,Parada obrigatório em Havana para tomar o icôn...,pt


In [9]:
# Revisa la información del dataset
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7875 entries, 0 to 7874
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  7875 non-null   object
 1   Class   7875 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 123.2+ KB


Nótese que no existe ningún dato faltante, también que todos los elementos de la clase son del tipo entero, por lo que no hay errores en la variable numérica. 

In [10]:
# Distribución de la columna Class
dataset['Class'].value_counts()

Class
5    2350
4    1971
3    1568
2    1173
1     813
Name: count, dtype: int64

Nótese que la distribución de la información respecto a la columna clase está desbalanceada, entonces hay más datos clasificados en las clases más altas

In [11]:
# Revisa la información de las palabras en lo que respecta a la variable categórica.


analisis['NumPalabras']= [len(word) for word in dataset['Review']]
analisis['Moda'] = analisis['Review'].apply(lambda review: mode([len(word) for word in review.split()]) if review else None)
analisis['Max'] = [max([len(word) for word in review.split(' ')]) for review in analisis['Review']]
analisis['Min'] = [min([len(word) for word in review.split(' ')]) for review in analisis['Review']]

analisis

,Review,Class,lenguaje,NumPalabras,Moda,Max,Min
0,Nos alojamos en una casa alquilada en la ciuda...,4,es,416,2,13,1
1,"La comida está bien, pero nada especial. Yo te...",3,es,263,2,14,1
2,"En mi opinión, no es una como muchos usuarios ...",3,es,612,3,16,1
3,esta curiosa forma que asemeja una silla de mo...,4,es,180,2,9,2
4,Lo mejor era la limonada. Me gusto la comida d...,2,es,88,2,9,1
...,...,...,...,...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,es,624,2,13,0
7871,Es difícil revisar el castillo porque apenas p...,3,es,609,2,15,1
7872,Si vas a Mérida no puedes perderte de este lug...,5,es,168,3,9,1
7873,"Este imperdible sitio, que lleva el nombre del...",5,es,424,2,18,0


# Preparación de los datos

En la preparación de los datos se van a realizar los siguientes pasos:
* Traducción de datos 
* Limpieza de datos
* Tokenización
* Normalización

In [12]:
# Crea un dataset diferente para hacer la preparación de los datos
prepData= analisis.copy()

## Traducción de los datos ajenos al idioma

In [13]:
#Se traducen los idiomas ajenos al lenguaje español
translator = Translator()
def translate_to_spanish(text):
    # Detecta el idioma del texto
    detected = translator.detect(text)
    
    # Verifica si el texto ya está en español
    if detected.lang == 'es':
        return text
    
    # Si el texto no está en español, lo traduce al español
    translated = translator.translate(text, src=detected.lang, dest='es')
    return translated.text
if 'lenguaje' in prepData.columns:
    prepData.loc[prepData['lenguaje'] == 'pt', 'Review'] = prepData.loc[prepData['lenguaje'] == 'pt', 'Review'].apply(lambda x: translate_to_spanish(x))
    prepData.loc[prepData['lenguaje'] == 'en', 'Review'] = prepData.loc[prepData['lenguaje'] == 'en', 'Review'].apply(lambda x: translate_to_spanish(x))
    portuguese_reviews = prepData[prepData['lenguaje'] == 'pt']
    portuguese_reviews[['Review', 'lenguaje']]
    #Verifica que se haya traducido el portugués
    print(portuguese_reviews[['Review', 'lenguaje']])
    english_reviews = prepData[prepData['lenguaje'] == 'en']
    #Verifica que se haya traducido el inglés
    print(english_reviews[['Review', 'lenguaje']])
    prepData.drop('lenguaje', axis=1, inplace=True)

                                                 Review lenguaje
1932  Parada obligatoria en La Habana para tomar la ...       pt
                                                 Review lenguaje
743   Una visita obligada cuando estás en Bogotá y e...       en
1497  La primera vez en Colombia y el hotel y el ser...       en
3368  Mis expectativas sobre este hotel fueron altas...       en
4835  Este mercado es como todos los mercados que he...       en


## Limpieza de datos

Para la limpieza de datos se va a garantizar que :
* Se deja todo en minuscula
* Remover carácteres no deseados
* Remover puntuación
* Manejar valores numéricos
* Remover palabras que no tienen un gran sentimiento (StopWords)

In [14]:
## Se deja todo en minuscula
def lower_case_words(tokenized_words):
    """Convert all words in a list to lowercase."""
    # Use a list comprehension to convert each word in the list to lowercase
    lowercased_words = [word.lower() for word in tokenized_words]
    
    return lowercased_words

In [15]:
# Elimina todos los elementos que no son necesarios y no se buscan utilizar

def remove_unwanted_characters_from_tokens(tokenized_words):
    """Remove unwanted non-ASCII characters from a list of tokenized words, retaining Spanish characters."""
    # Include the Spanish alphabet, both lowercase and uppercase, digits, and common punctuation
    allowed_characters = set("abcdefghijklmnopqrstuvwxyzáéíóúüñABCDEFGHIJKLMNOPQRSTUVWXYZÁÉÍÓÚÜÑ " + string.digits + string.punctuation)
    
    # Initialize an empty list to hold the cleaned words
    new_words = []
    
    # Iterate over each word in the list of tokenized words
    for word in tokenized_words:
        # Filter each word to include only allowed characters
        cleaned_word = ''.join(char for char in word if char in allowed_characters)
        # Append the cleaned word to the new list
        if cleaned_word == word:
            # If they are the same, it means the word contains only allowed characters, so append it to the list
            new_words.append(cleaned_word)
    
    return new_words

In [16]:
# Remueve la punctuación dentro de cada palabra
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words, with explicit support for Unicode characters."""
    new_words = []
    for word in words:
        if word is not None:
            # Use \w for word characters, \s for whitespace, and ensure Unicode support
            new_word = re.sub(r'[^\w\s]', '', word, flags=re.UNICODE)
            if new_word != '':
                new_words.append(new_word)
    return new_words

In [17]:
# Convierte los números a palabras
def convert_numbers_to_words(words):
    """Convert all numbers in a text to their word representation in the specified language."""
    
    # Convert each word
    new_words = []
    for word in words:
        if word.isdigit():
            word= float(word)
            # Convert the numerical word to its word representation
            word_in_text = num2words(word, lang='es')
            new_words.append(word_in_text)
        else:
            new_words.append(word)
    return new_words

In [18]:
# Remueve las stopwords o palabras sin mucha prioridad en el analisis
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    # Filter out any words that are in the list of stop words
    filtered_words = [word for word in words if word not in spanish_stopwords]
    return filtered_words

In [19]:
def preprocessing(words):
    """Preprocess a list of tokenized words"""
    # Remove unwanted characters
    words = remove_unwanted_characters_from_tokens(words)
    # Convert to lowercase
    words = lower_case_words(words)
    # Convert numbers to words
    words = convert_numbers_to_words(words)
    # Remove punctuation
    words = remove_punctuation(words)
    # Remove stop words
    words = remove_stopwords(words)
    return words

## Tokenización

In [20]:
#Expandir las contracciones y reemplazarlas
def expandir_contracciones(texto, contracciones_dict):
    """
    Expande las contracciones encontradas en el texto basándose en un diccionario de contracciones.
    """
    # Asegurarse de que la expresión regular coincide solo con palabras completas. 
    # \b es un límite de palabra, lo que ayuda a coincidir solo palabras completas.
    contracciones_re = re.compile(r'\b(%s)\b' % '|'.join(map(re.escape, contracciones_dict.keys())))

    def reemplazo(match):
        return contracciones_dict[match.group(0)]

    texto_expandido = contracciones_re.sub(reemplazo, texto)
    
    return texto_expandido

prepData['Review'] = prepData['Review'].apply(expandir_contracciones, contracciones_dict=contracciones)
prepData

,Review,Class,NumPalabras,Moda,Max,Min
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1
...,...,...,...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,624,2,13,0
7871,Es difícil revisar el castillo porque apenas p...,3,609,2,15,1
7872,Si vas a Mérida no puedes perderte de este lug...,5,168,3,9,1
7873,"Este imperdible sitio, que lleva el nombre de ...",5,424,2,18,0


In [35]:
#Tokenizar
def nltk_spanish_tokenizer(text):
    return word_tokenize(text, language='spanish')
prepData['words'] = prepData['Review'].apply(lambda x: word_tokenize(x, language='spanish'))
prepData.head()


,Review,Class,NumPalabras,Moda,Max,Min,words,words1,words2
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1,"[Nos, alojamos, en, una, casa, alquilada, en, ...","[alojamos, casa, alquilada, ciudad, amurallada...","[aloj, cas, alquil, ciud, amurall, parec, tan,..."
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1,"[La, comida, está, bien, ,, pero, nada, especi...","[comida, bien, especial, mejor, comida, mexcan...","[com, bien, especial, mejor, com, mexc, unid, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1,"[En, mi, opinión, ,, no, es, una, como, muchos...","[opinión, usuarios, reclaman, gran, paladar, p...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2,"[esta, curiosa, forma, que, asemeja, una, sill...","[curiosa, forma, asemeja, silla, montar, ahi, ...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...","[mejor, limonada, gusto, comida, mundo, sosa, ...","[mejor, limon, gust, com, mund, sos, fri]"


In [36]:
#Elimina los valores nulos del conjunto de palabras
prepData['words'].dropna()

0       [Nos, alojamos, en, una, casa, alquilada, en, ...
1       [La, comida, está, bien, ,, pero, nada, especi...
2       [En, mi, opinión, ,, no, es, una, como, muchos...
3       [esta, curiosa, forma, que, asemeja, una, sill...
4       [Lo, mejor, era, la, limonada, ., Me, gusto, l...
                              ...                        
7870    [El, motivo, de, mi, estancia, fue, porque, vi...
7871    [Es, difícil, revisar, el, castillo, porque, a...
7872    [Si, vas, a, Mérida, no, puedes, perderte, de,...
7873    [Este, imperdible, sitio, ,, que, lleva, el, n...
7874    [Festejando, Dia, de, el, Amor, y, Amistad, Te...
Name: words, Length: 7875, dtype: object

In [37]:
prepData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7875 entries, 0 to 7874
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review       7875 non-null   object
 1   Class        7875 non-null   int64 
 2   NumPalabras  7875 non-null   int64 
 3   Moda         7875 non-null   int64 
 4   Max          7875 non-null   int64 
 5   Min          7875 non-null   int64 
 6   words        7875 non-null   object
 7   words1       7875 non-null   object
 8   words2       7875 non-null   object
dtypes: int64(5), object(4)
memory usage: 553.8+ KB


In [38]:
#Aplica el procesamiento básico
prepData['words1']=prepData['words'].apply(preprocessing)
prepData.head()

,Review,Class,NumPalabras,Moda,Max,Min,words,words1,words2
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1,"[Nos, alojamos, en, una, casa, alquilada, en, ...","[alojamos, casa, alquilada, ciudad, amurallada...","[aloj, cas, alquil, ciud, amurall, parec, tan,..."
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1,"[La, comida, está, bien, ,, pero, nada, especi...","[comida, bien, especial, mejor, comida, mexcan...","[com, bien, especial, mejor, com, mexc, unid, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1,"[En, mi, opinión, ,, no, es, una, como, muchos...","[opinión, usuarios, reclaman, gran, paladar, p...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2,"[esta, curiosa, forma, que, asemeja, una, sill...","[curiosa, forma, asemeja, silla, montar, ahi, ...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...","[mejor, limonada, gusto, comida, mundo, sosa, ...","[mejor, limon, gust, com, mund, sos, fri]"


## Normalización

In [39]:
#Stemmer

stemmer = SnowballStemmer('spanish')
def stem_words(words):
    """Stem words in list of tokenized words"""
    # Stem each word in the list of words
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

#This is just for trying the stemmer
def stemExample(text):
   return [stemmer.stem(word) for word in text.split()]

In [40]:
## Apply Stemmer to the words
prepData['words2'] = prepData['words1'].apply(stem_words)
prepData.head()

,Review,Class,NumPalabras,Moda,Max,Min,words,words1,words2
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1,"[Nos, alojamos, en, una, casa, alquilada, en, ...","[alojamos, casa, alquilada, ciudad, amurallada...","[aloj, cas, alquil, ciud, amurall, parec, tan,..."
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1,"[La, comida, está, bien, ,, pero, nada, especi...","[comida, bien, especial, mejor, comida, mexcan...","[com, bien, especial, mejor, com, mexc, unid, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1,"[En, mi, opinión, ,, no, es, una, como, muchos...","[opinión, usuarios, reclaman, gran, paladar, p...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2,"[esta, curiosa, forma, que, asemeja, una, sill...","[curiosa, forma, asemeja, silla, montar, ahi, ...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...","[mejor, limonada, gusto, comida, mundo, sosa, ...","[mejor, limon, gust, com, mund, sos, fri]"


In [41]:
# #Lemmatization

# # Initialize the Spanish pipeline
# nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

# def lemmatize_words(words):
#     """Lemmatize a list of words in Spanish"""
#     # Join words into a single string as Stanza processes text inputs
#     text = ' '.join(words)
    
#     # Process the text
#     doc = nlp(text)
    
#     # Extract lemmas for each word
#     lemmatized_words = [word.lemma for sentence in doc.sentences for word in sentence.words]
#     return lemmatized_words

In [42]:
# ## Apply lemmatization to the words
# prepData['words3'] = prepData['words1'].apply(lemmatize_words)
# prepData.head()

In [43]:
prepData

,Review,Class,NumPalabras,Moda,Max,Min,words,words1,words2
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1,"[Nos, alojamos, en, una, casa, alquilada, en, ...","[alojamos, casa, alquilada, ciudad, amurallada...","[aloj, cas, alquil, ciud, amurall, parec, tan,..."
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1,"[La, comida, está, bien, ,, pero, nada, especi...","[comida, bien, especial, mejor, comida, mexcan...","[com, bien, especial, mejor, com, mexc, unid, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1,"[En, mi, opinión, ,, no, es, una, como, muchos...","[opinión, usuarios, reclaman, gran, paladar, p...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2,"[esta, curiosa, forma, que, asemeja, una, sill...","[curiosa, forma, asemeja, silla, montar, ahi, ...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...","[mejor, limonada, gusto, comida, mundo, sosa, ...","[mejor, limon, gust, com, mund, sos, fri]"
...,...,...,...,...,...,...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,624,2,13,0,"[El, motivo, de, mi, estancia, fue, porque, vi...","[motivo, estancia, vine, congreso, medico, hos...","[motiv, estanci, vin, congres, medic, hosped, ..."
7871,Es difícil revisar el castillo porque apenas p...,3,609,2,15,1,"[Es, difícil, revisar, el, castillo, porque, a...","[difícil, revisar, castillo, apenas, podíamos,...","[dificil, revis, castill, apen, pod, camin, so..."
7872,Si vas a Mérida no puedes perderte de este lug...,5,168,3,9,1,"[Si, vas, a, Mérida, no, puedes, perderte, de,...","[si, vas, mérida, puedes, perderte, lugar, nue...","[si, vas, mer, pued, perdert, lug, nuev, sucur..."
7873,"Este imperdible sitio, que lleva el nombre de ...",5,424,2,18,0,"[Este, imperdible, sitio, ,, que, lleva, el, n...","[imperdible, sitio, lleva, nombre, conquistado...","[imperd, siti, llev, nombr, conquist, joy, urb..."


# Creación e implementación de modelos

In [44]:
## Unificación de las palabras
molData=prepData.copy()
molData['words'] = molData['words2'].apply(lambda x: ' '.join(map(str, x)))
molData

,Review,Class,NumPalabras,Moda,Max,Min,words,words1,words2
0,Nos alojamos en una casa alquilada en la ciuda...,4,416,2,13,1,aloj cas alquil ciud amurall parec tan segur c...,"[alojamos, casa, alquilada, ciudad, amurallada...","[aloj, cas, alquil, ciud, amurall, parec, tan,..."
1,"La comida está bien, pero nada especial. Yo te...",3,263,2,14,1,com bien especial mejor com mexc unid margarit...,"[comida, bien, especial, mejor, comida, mexcan...","[com, bien, especial, mejor, com, mexc, unid, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,612,3,16,1,opinion usuari reclam gran palad parec ser par...,"[opinión, usuarios, reclaman, gran, paladar, p...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,180,2,9,2,curios form asemej sill mont ahi nombr icon ci...,"[curiosa, forma, asemeja, silla, montar, ahi, ...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,88,2,9,1,mejor limon gust com mund sos fri,"[mejor, limonada, gusto, comida, mundo, sosa, ...","[mejor, limon, gust, com, mund, sos, fri]"
...,...,...,...,...,...,...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,624,2,13,0,motiv estanci vin congres medic hosped lug ins...,"[motivo, estancia, vine, congreso, medico, hos...","[motiv, estanci, vin, congres, medic, hosped, ..."
7871,Es difícil revisar el castillo porque apenas p...,3,609,2,15,1,dificil revis castill apen pod camin sofoc cal...,"[difícil, revisar, castillo, apenas, podíamos,...","[dificil, revis, castill, apen, pod, camin, so..."
7872,Si vas a Mérida no puedes perderte de este lug...,5,168,3,9,1,si vas mer pued perdert lug nuev sucursal ampl...,"[si, vas, mérida, puedes, perderte, lugar, nue...","[si, vas, mer, pued, perdert, lug, nuev, sucur..."
7873,"Este imperdible sitio, que lleva el nombre de ...",5,424,2,18,0,imperd siti llev nombr conquist joy urbanasu a...,"[imperdible, sitio, lleva, nombre, conquistado...","[imperd, siti, llev, nombr, conquist, joy, urb..."


In [130]:
## Crea el test y el train set

X_train, X_test, y_train, y_test = train_test_split(molData['words'], molData['Class'], test_size=0.2, random_state=42)

## Regresión logistica

In [131]:
vectorizer = CountVectorizer()
X_train_transformed  = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)
print(X_train_transformed .shape)


(6300, 12922)


In [132]:
tfid= TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
X_train_transformed=tfid.fit_transform(X_train_transformed)
X_test_transformed=tfid.transform(X_test_transformed)

In [117]:
# Initialize and train logistic regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_transformed, y_train)

# Predictions on test data
y_pred = logistic_model.predict(X_test_transformed)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4653968253968254


In [124]:
svm = LinearSVC(random_state=42)
svm.fit(X_train_transformed, y_train)
y_pred_svm_sample = svm.predict(X_test_transformed)
accuracy_svm = accuracy_score(y_test, y_pred_svm_sample)
print("Accuracy:", accuracy_svm)

c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.43682539682539684


c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [125]:
# Initialize and train Random Forest on the reduced dataset
random_forest_sample = RandomForestClassifier(random_state=42)
random_forest_sample.fit(X_train_transformed, y_train)

# Predict on the test set and calculate accuracy
y_pred_rf_sample = random_forest_sample.predict(X_test_transformed)
accuracy_rf_sample = accuracy_score(y_test, y_pred_rf_sample)

print("Accuracy:",accuracy_rf_sample)

Accuracy: 0.4685714285714286


In [135]:
# Initialize the neural network model
# Let's use a simple architecture with two hidden layers of 100 neurons each
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=300, activation='relu', solver='adam', random_state=42)

# Train the neural network on the reduced dataset
mlp.fit(X_train_transformed, y_train)

# Predict on the test set and calculate accuracy
y_pred_mlp = mlp.predict(X_test_transformed)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

accuracy_mlp

0.433015873015873

In [137]:
#DecisionTree
# Initialize the Decision Tree Regressor
decision_tree_regressor = DecisionTreeRegressor(random_state=42)

# Train the regressor on the reduced dataset
decision_tree_regressor.fit(X_train_transformed, y_train)

# Predict on the test set
y_pred_dtr = decision_tree_regressor.predict(X_test_transformed)

# For regression tasks, we typically use different metrics than accuracy, such as Mean Absolute Error (MAE)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred_dtr)

### Menos es mejor
print(mae)

0.9574603174603175


In [55]:
# tfidf = TfidfVectorizer(strip_accents=None,
#                          lowercase=False,
#                          preprocessor=None)
# param_grid = [{'vect__ngram_range': [(1,1)],
#                 'vect__stop_words': [spanish_stopwords, None],
#                 'vect__tokenizer': [nltk_spanish_tokenizer],
#                 'clf__penalty': ['l1', 'l2'],
#                 'clf__C': [1.0, 10.0, 100.0]},
#                {'vect__ngram_range': [(1,1)],
#                 'vect__stop_words': [spanish_stopwords, None],
#                 'vect__tokenizer': [nltk_spanish_tokenizer],
#                 'vect__use_idf':[False],
#                 'vect__norm':[None],
#                 'clf__penalty': ['l1', 'l2'],
#                 'clf__C': [1.0, 10.0, 100.0]}
#               ]
# lr_tfidf = Pipeline([('vect', tfidf),
#                       ('clf',
#                        LogisticRegression(random_state=0,
#                                           solver='liblinear'))])
# gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
#                             scoring='accuracy',
#                             cv=5, verbose=2,
#                             n_jobs=1)
 
# gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words={'suyas', 'tenemos', 'sois', 'hemos', 'fuiste', 'hubo', 'del', 'uno', 'tiene', 'este', 'seremos', 'y', 'nosotras', 'serás', 'tuviéramos', 'al', 'vosotros', 'estuviéramos', 'sentida', 'hayas', 'erais', 'ese', 'tendréis', 'estábamos', 'estéis', 'mías', 'sería', 'nuestro', 'estuvo', 'estuviera', 'fue', 'estaría', 'habréis', 'tienes', 'estuve', 'tendrá', 'habrías', 'tuya', 'algunos', 'estamos', 'seríamos', 'hubierais', 'lo', 'algunas', 'donde', 'el', 'tenidos', 'como', 'tenida', 'tenga', 'contra', 'esté', 'hubieron', 'habrían', 'estuvieses', 'antes', 'les', 'tengan', 'suyos', 'estando', 'tened', 'esa', 'sobre', 'habrán', 'hubiesen', 'tendremos', 'estaba', 'tenía', 'seamos', 'estuvieran', 'estarías', 'estuvisteis', 'tuviesen', 'estabais', 'estaremos', 'algo', 'otras', 'tenidas', 'habían', 'estada', 'eras', 'os', 'o', 'estabas', 'las', 'tendrías', 'ha

c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function nltk_spanish_tokenizer at 0x000001F7423ED9D0>; total time=   5.2s


c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function nltk_spanish_tokenizer at 0x000001F7423ED9D0>; total time=   5.2s


...

[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function nltk_spanish_tokenizer at 0x000001F7423ED9D0>, vect__use_idf=False; total time=   9.0s


c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=100.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function nltk_spanish_tokenizer at 0x000001F7423ED9D0>, vect__use_idf=False; total time=   9.0s


c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\esteb\AppData\Local\Programs\Python\Python39\lib\site-packages\s

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=0,
                                                           solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [{'a', 'al', 'algo', 'algunas',
                                                'algunos', 'ante', 'antes',
                                                'como', 'con', 'contra', 'cual',
                                                'cuando', 'd...
                          'vect__stop_words': [{'a', 'al', 'algo', 'algunas',
                                                'algunos', 'ante', 'antes',
                                                'como', 'con', 'contra', 'cual',
                                                'cuando', 'de', 'del', 'desde',
                                                'donde', 'durante', 'e', 'el',
                                                'ella', 'ellas', 'ellos', 'en',
                                                'entre', 'era', 'erais', 'eran',
                                                'eras', 'eres', 'es', ...},
                                               None],
                          'vect__tokenizer': [<function nltk_spanish_tokenizer at 0x000001F7423ED9D0>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [56]:
# print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)


Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function nltk_spanish_tokenizer at 0x000001F7423ED9D0>} 


In [57]:
# print('CV Accuracy: %.3f'      % gs_lr_tfidf.best_score_)

CV Accuracy: 0.487


In [ ]:
clf = gs_lr_tfidf.best_estimator_
 